# データサイエンス100本ノック（構造化データ加工編） - R

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（!マークに続けてOSコマンドを入力することで、任意のubuntu Linuxコマンドが入力可能）
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
require('RPostgreSQL')
require('tidyr')
require('dplyr')
require('stringr')
require('caret')
require('lubridate')
require('rsample')
require('recipes')

host <- 'db'
port <- Sys.getenv()["PG_PORT"]
dbname <- Sys.getenv()["PG_DATABASE"]
user <- Sys.getenv()["PG_USER"]
password <- Sys.getenv()["PG_PASSWORD"]

con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
df_customer <- dbGetQuery(con,"SELECT * FROM customer")
df_category <- dbGetQuery(con,"SELECT * FROM category")
df_product <- dbGetQuery(con,"SELECT * FROM product")
df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
df_store <- dbGetQuery(con,"SELECT * FROM store")
df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

Loading required package: RPostgreSQL

Loading required package: DBI

Loading required package: tidyr

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: stringr

Loading required package: caret

Loading required package: lattice

Loading required package: ggplot2

Loading required package: lubridate


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading required package: rsample

Loading required package: recipes


Attaching package: ‘recipes’


The following object is masked from ‘package:stringr’:

    fixed


The following object is masked from ‘package:stats’:

    step




# 演習問題

---
> R-001: レシート明細データフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [2]:
head(df_receipt, n = 10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
7,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
8,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
9,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770


---
> R-002: レシート明細データフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [3]:
head(df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")], n=10)

,sales_ymd,customer_id,product_cd,amount
,<int>,<chr>,<chr>,<int>
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> R-003: レシート明細データフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [4]:
head(rename(df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")], sales_date = sales_ymd), n=10)

,sales_date,customer_id,product_cd,amount
,<int>,<chr>,<chr>,<int>
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> R-004: レシート明細データフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [5]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001")

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102
20180414,CS018205000001,P071101002,278
20190226,CS018205000001,P070902035,168
20190924,CS018205000001,P060805001,495
20190226,CS018205000001,P071401020,2200


---
> R-005: レシート明細データフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [6]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001" & amount >= 1000)

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401012,2200
20190226,CS018205000001,P071401020,2200
20180911,CS018205000001,P071401005,1100


---
> R-006: レシート明細データフレーム（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [7]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "quantity", "amount")] %>%
    filter(customer_id == "CS018205000001" &
           (amount >= 1000 | quantity >= 5))

sales_ymd,customer_id,product_cd,quantity,amount
<int>,<chr>,<chr>,<int>,<int>
20180911,CS018205000001,P071401012,1,2200
20180414,CS018205000001,P060104007,6,600
20170614,CS018205000001,P050206001,5,990
20190226,CS018205000001,P071401020,1,2200
20180911,CS018205000001,P071401005,1,1100


---
> R-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [8]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001" &
           between(amount, 1000, 2000))

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401005,1100


---
> R-008: レシート明細データフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [9]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001" &
           product_cd != "P071401019")

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102
20180414,CS018205000001,P071101002,278
20190226,CS018205000001,P070902035,168
20190924,CS018205000001,P060805001,495
20190226,CS018205000001,P071401020,2200


---
> R-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`
df_store %>%
    filter(!(prefecture_cd == "13" | floor_area > 900))
`

In [10]:
df_store %>%
    filter(prefecture_cd != "13" & floor_area <= 900)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890
S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808


---
> R-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [11]:
head(df_store %>% filter(grepl("^S14", store_cd)), n=10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
2,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
3,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679
4,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
5,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830
6,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574
7,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341
8,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831
9,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838


---
> R-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [12]:
head(df_customer %>% filter(grepl("1$", customer_id)), n=10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
3,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
4,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
5,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
6,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
7,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
8,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
9,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0


---
> R-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [13]:
df_store %>% filter(grepl("横浜市", address))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890
S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657


---
> R-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [14]:
head(df_customer %>% filter(grepl("^[A-F]", status_cd)), n=10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
2,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
5,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
6,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
7,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
8,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
9,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E


---
> R-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [15]:
head(df_customer %>% filter(grepl("[1-9]$", status_cd)), n=10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
2,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
5,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
6,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
7,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
8,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
9,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1


---
> R-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [16]:
head(df_customer %>% filter(grepl("^[A-F].*[1-9]$", status_cd)), n=10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
2,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
3,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
4,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
5,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
6,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
7,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
8,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
9,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1


---
> R-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [17]:
df_store %>% filter(grepl("^[0-9]{3}-[0-9]{3}-[0-9]{4}$", tel_no))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830
S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574


---
> R-017: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [18]:
head(
    df_customer[order(df_customer$birth_day
                      , decreasing = FALSE), ],
    n=10
)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
18818,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12329,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15683,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15303,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1682,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7512,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2379,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4681,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
6306,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> R-018: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [19]:
head(
    df_customer[order(df_customer$birth_day,
                      decreasing = TRUE), ],
    n=10
)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
15640,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7469,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10746,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19812,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7040,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3671,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12494,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15978,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5717,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F


---
> R-019: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [20]:
df_receipt %>%
    mutate(ranking = min_rank(desc(amount))) %>%
    arrange(ranking) %>%
    slice(1:10) 

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,ranking
<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>
20190903,1567468800,S14011,1192,2,CS011415000006,P080104055,1,10925,1
20190304,1551657600,S14046,1152,2,ZZ000000000000,P090302118,1,6800,2
20170921,1505952000,S14028,1192,2,CS028605000002,P090907210,1,5780,3
20170721,1500595200,S13015,1112,1,CS015515000034,P090907208,1,5480,4
20190103,1546473600,S14021,1162,2,ZZ000000000000,P090907208,1,5480,4
20190724,1563926400,S13009,1192,2,ZZ000000000000,P090907209,1,5480,4
20180621,1529539200,S13004,1172,2,ZZ000000000000,P090907206,1,5440,7
20190209,1549670400,S14021,1142,2,CS021515000089,P090907206,1,5440,7
20181219,1545177600,S13015,1132,1,CS015515000083,P071301009,1,5280,9


---
> R-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [21]:
df_receipt %>%
    mutate(ranking = row_number(desc(amount))) %>%
    arrange(ranking) %>%
    slice(1:10) 

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,ranking
<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>
20190903,1567468800,S14011,1192,2,CS011415000006,P080104055,1,10925,1
20190304,1551657600,S14046,1152,2,ZZ000000000000,P090302118,1,6800,2
20170921,1505952000,S14028,1192,2,CS028605000002,P090907210,1,5780,3
20170721,1500595200,S13015,1112,1,CS015515000034,P090907208,1,5480,4
20190103,1546473600,S14021,1162,2,ZZ000000000000,P090907208,1,5480,5
20190724,1563926400,S13009,1192,2,ZZ000000000000,P090907209,1,5480,6
20180621,1529539200,S13004,1172,2,ZZ000000000000,P090907206,1,5440,7
20190209,1549670400,S14021,1142,2,CS021515000089,P090907206,1,5440,8
20181219,1545177600,S13015,1132,1,CS015515000083,P071301009,1,5280,9


---
> R-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [22]:
nrow(df_receipt)

[1] 104681

---
> R-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [23]:
length(unique(df_receipt$customer_id))

[1] 8307

---
> R-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [24]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(amount=sum(amount),
              quantity=sum(quantity), .groups = 'drop')

store_cd,amount,quantity
<chr>,<int>,<int>
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403
S13001,811936,2347
S13002,727821,2340
S13003,764294,2197
S13004,779373,2390


---
> R-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [25]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(max_ymd=max(sales_ymd), .groups = 'drop') %>%
    slice(1:10)

customer_id,max_ymd
<chr>,<int>
CS001113000004,20190308
CS001114000005,20190731
CS001115000010,20190405
CS001205000004,20190625
CS001205000006,20190224
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906
CS001212000046,20170811


---
> R-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [26]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(min_ymd=min(sales_ymd), .groups = 'drop') %>%
    slice(1:10)

customer_id,min_ymd
<chr>,<int>
CS001113000004,20190308
CS001114000005,20180503
CS001115000010,20171228
CS001205000004,20170914
CS001205000006,20180207
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906
CS001212000046,20170811


---
> R-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [27]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(max_ymd=max(sales_ymd)
             ,min_ymd=min(sales_ymd), .groups = 'drop') %>%
    filter(max_ymd != min_ymd) %>%
    slice(1:10)

customer_id,max_ymd,min_ymd
<chr>,<int>,<int>
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207
CS001214000009,20190902,20170306
CS001214000017,20191006,20180828
CS001214000048,20190929,20171109
CS001214000052,20190617,20180208
CS001215000005,20181021,20170206


---
> R-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [28]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(mean_amount=mean(amount), .groups = 'drop') %>%
    arrange(desc(mean_amount)) %>%
    slice(1:5)

store_cd,mean_amount
<chr>,<dbl>
S13052,402.8675
S13015,351.1120
S13003,350.9155
S14010,348.7913
S13001,348.4704


---
> R-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [29]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(median_amount=median(amount), .groups = 'drop') %>%
    arrange(desc(median_amount)) %>%
    slice(1:5)

store_cd,median_amount
<chr>,<dbl>
S13052,190
S14010,188
S14050,185
S13003,180
S13018,180


---
> R-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [30]:
# コード例1
df_receipt %>%
  group_by(store_cd, product_cd) %>%
  summarise(n = n(), .groups = 'drop_last') %>%
  filter(n == n %>% max()) %>%
  ungroup()

store_cd,product_cd,n
<chr>,<chr>,<int>
S12007,P060303001,72
S12013,P060303001,107
S12014,P060303001,65
S12029,P060303001,92
S12030,P060303001,115
S13001,P060303001,67
S13002,P060303001,78
S13003,P071401001,65
S13004,P060303001,88


In [31]:
# コード例2：whtch.max()を使ってこんな感じも（最頻値複数発生時、どれか一つに絞られる）
table_product <-table(c(df_receipt$store_cd), df_receipt$product_cd)

store <- names(table_product[,1])

mode_product <- c()
for (i in 1:length(store)){
    mode_product[i] <- names(which.max(table_product[i,]))
}

data.frame(store_cd = store, store_cd = mode_product)

store_cd,store_cd.1
<fct>,<fct>
S12007,P060303001
S12013,P060303001
S12014,P060303001
S12029,P060303001
S12030,P060303001
S13001,P060303001
S13002,P060303001
S13003,P071401001
S13004,P060303001


---
> R-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [32]:
var_sample <- function(x){ var(x)*(length(x)-1)/length(x) }


df_receipt %>%
    group_by(store_cd) %>%
    summarise(var_amount=var_sample(amount), .groups = 'drop') %>%
    arrange(desc(var_amount)) %>%
    slice(1:5)

store_cd,var_amount
<chr>,<dbl>
S13052,440088.7
S14011,306314.6
S14034,296920.1
S13001,295432.0
S13015,295294.4


---
> R-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [33]:
var_sample <- function(x){ var(x)*(length(x)-1)/length(x) }
std_sample <- function(x){ sqrt(var_sample(x)) }

head(
    df_receipt %>%
        group_by(store_cd) %>%
        summarise(std_amount=std_sample(amount), .groups = 'drop') %>%
        arrange(desc(std_amount))
    , n=5
)

store_cd,std_amount
<chr>,<dbl>
S13052,663.3918
S14011,553.4569
S14034,544.9037
S13001,543.5366
S13015,543.4099


---
> R-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [34]:
df_receipt %>%
    summarise(amount_25per=quantile(amount, 0.25),
              amount_50per=quantile(amount, 0.5),
              amount_75per=quantile(amount, 0.75),
              amount_100per=quantile(amount, 1.0))

amount_25per,amount_50per,amount_75per,amount_100per
<dbl>,<dbl>,<dbl>,<dbl>
102,170,288,10925


---
> R-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [35]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(mean_amount=mean(amount), .groups = 'drop') %>%
    filter(mean_amount >= 330)

store_cd,mean_amount
<chr>,<dbl>
S12013,330.1941
S13001,348.4704
S13003,350.9155
S13004,330.9439
S13015,351.1120
S13019,330.2086
S13020,337.8799
S13052,402.8675
S14010,348.7913


---
> R-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [36]:
df_mean <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount=sum(amount), .groups = 'drop') %>%
    summarise(mean_amount=mean(sum_amount))
df_mean$mean_amount

[1] 2547.742

---
> R-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [37]:
df_sum <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount=sum(amount), .groups = 'drop')
df_mean <- df_sum %>%
    summarise(mean_amount=mean(sum_amount))
df_sum %>%
    filter(sum_amount > df_mean$mean_amount) %>%
    slice(1:10)

customer_id,sum_amount
<chr>,<int>
CS001115000010,3044
CS001205000006,3337
CS001214000009,4685
CS001214000017,4132
CS001214000052,5639
CS001215000040,3496
CS001304000006,3726
CS001305000005,3485
CS001305000011,4370


---
> R-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [38]:
inner_join(df_receipt, df_store[c('store_cd', 'store_name')], by = 'store_cd') %>%
    slice(1:10)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>
20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,成城店
20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,二ツ橋店
20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,新山下店
20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,大和店
20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138,狛江店
20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30,三田店
20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128,長津田店
20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770,十条仲原店


---
> R-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [39]:
inner_join(df_product,
           df_category[c('category_small_cd','category_small_name')],
           by = 'category_small_cd') %>%
    slice(1:10)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>
P040101001,04,0401,040101,198,149,弁当類
P040101002,04,0401,040101,218,164,弁当類
P040101003,04,0401,040101,230,173,弁当類
P040101004,04,0401,040101,248,186,弁当類
P040101005,04,0401,040101,268,201,弁当類
P040101006,04,0401,040101,298,224,弁当類
P040101007,04,0401,040101,338,254,弁当類
P040101008,04,0401,040101,420,315,弁当類
P040101009,04,0401,040101,498,374,弁当類


---
> R-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [40]:
df_sum <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount=sum(amount), .groups = 'drop')

df_target <- df_customer %>%
    filter(!grepl("^Z", customer_id) & gender_cd == '1')

left_join(df_target['customer_id'], df_sum, by = 'customer_id') %>%
    replace_na(list(sum_amount=0)) %>%
    slice(1:10)

customer_id,sum_amount
<chr>,<dbl>
CS021313000114,0
CS031415000172,5088
CS028811000001,0
CS001215000145,875
CS015414000103,3122
CS033513000180,868
CS035614000014,0
CS011215000048,3444
CS009413000079,0


---
> R-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [41]:
# コード例１
df_sum <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount=sum(amount), .groups = 'drop') %>%
    arrange(desc(sum_amount)) %>%
    slice(1:20)

df_cnt <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(come_days=n_distinct(sales_ymd), .groups = 'drop') %>%
    arrange(desc(come_days), customer_id) %>%
    slice(1:20)

full_join(df_sum, df_cnt, by = "customer_id")

customer_id,sum_amount,come_days
<chr>,<int>,<int>
CS017415000097,23086,20
CS015415000185,20153,22
CS031414000051,19202,19
CS028415000007,19127,21
CS001605000009,18925,NA
CS010214000010,18585,22
CS006515000023,18372,NA
CS016415000141,18372,20
CS011414000106,18338,NA


In [42]:
# コード例2
# slice_maxを使用したコード例
# with_ties = FALSEをwith_ties = TRUEに書き換えると同一順位を含める形に変更できる

df_sum <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount)) %>%
    slice_max(sum_amount, n = 20, with_ties = FALSE)

df_cnt <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(come_days = n_distinct(sales_ymd)) %>%
    slice_max(come_days, n = 20, with_ties = FALSE)

df_original <- full_join(df_sum, df_cnt, by = "customer_id")

df_original

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)



customer_id,sum_amount,come_days
<chr>,<int>,<int>
CS017415000097,23086,20
CS015415000185,20153,22
CS031414000051,19202,19
CS028415000007,19127,21
CS001605000009,18925,NA
CS010214000010,18585,22
CS006515000023,18372,NA
CS016415000141,18372,20
CS011414000106,18338,NA


---
> R-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [43]:
df_store_tmp <- df_store
df_product_tmp <- df_product
df_store_tmp['key'] <- 0
df_product_tmp['key'] <- 0
nrow(full_join(df_store_tmp, df_product_tmp, by = "key"))

[1] 531590

---
> R-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [44]:
df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount=sum(amount), .groups = 'drop') %>%
    mutate(lag_ymd = lag(sales_ymd), lag_amount = lag(sum_amount), diff_amount = sum_amount - lag_amount) %>%
    slice(1:10)

sales_ymd,sum_amount,lag_ymd,lag_amount,diff_amount
<int>,<int>,<int>,<int>,<int>
20170101,33723,NA,NA,NA
20170102,24165,20170101,33723,-9558
20170103,27503,20170102,24165,3338
20170104,36165,20170103,27503,8662
20170105,37830,20170104,36165,1665
20170106,32387,20170105,37830,-5443
20170107,23415,20170106,32387,-8972
20170108,24737,20170107,23415,1322
20170109,26718,20170108,24737,1981


---
> R-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [45]:
# コード例1:縦持ちケース
df_sum <-df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount=sum(amount), .groups = 'drop')

for (i in 1:3){
    if (i == 1){
        df_lag <- df_sum %>%
            mutate(lag_ymd = lag(sales_ymd, n=i), lag_amount = lag(sum_amount, n=i))
    }
    else{
        df_tmp <- df_sum %>%
            mutate(lag_ymd = lag(sales_ymd, n=i), lag_amount = lag(sum_amount, n=i))
        df_lag <- rbind(df_lag, df_tmp)
    }
}

drop_na(df_lag, everything()) %>%
    arrange(sales_ymd, lag_ymd) %>%
    slice(1:10)

sales_ymd,sum_amount,lag_ymd,lag_amount
<int>,<int>,<int>,<int>
20170102,24165,20170101,33723
20170103,27503,20170101,33723
20170103,27503,20170102,24165
20170104,36165,20170101,33723
20170104,36165,20170102,24165
20170104,36165,20170103,27503
20170105,37830,20170102,24165
20170105,37830,20170103,27503
20170105,37830,20170104,36165


In [46]:
# コード例2:横持ちケース
df_sum <-df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount=sum(amount), .groups = 'drop')

for (i in 1:3){
    if (i == 1){
        df_lag <- df_sum %>%
            mutate(lag_ymd_1 = lag(sales_ymd, n=i), lag_amount_1 = lag(sum_amount, n=i))
    }
    else{
        col_name_1 <- paste("lag_ymd", i , sep="_")
        col_name_2 <- paste("lag_amount", i , sep="_")
        print(col_name_1)
        print(col_name_2)
        df_tmp <- df_sum %>%
            mutate(!!col_name_1 := lag(sales_ymd, n=i), !!col_name_2 := lag(sum_amount, n=i))
        df_lag <- cbind(df_lag, df_tmp[c(col_name_1, col_name_2)])
    }
}

drop_na(df_lag, everything()) %>%
    arrange(sales_ymd) %>%
    slice(1:10)

[1] "lag_ymd_2"
[1] "lag_amount_2"
[1] "lag_ymd_3"
[1] "lag_amount_3"


sales_ymd,sum_amount,lag_ymd_1,lag_amount_1,lag_ymd_2,lag_amount_2,lag_ymd_3,lag_amount_3
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
20170104,36165,20170103,27503,20170102,24165,20170101,33723
20170105,37830,20170104,36165,20170103,27503,20170102,24165
20170106,32387,20170105,37830,20170104,36165,20170103,27503
20170107,23415,20170106,32387,20170105,37830,20170104,36165
20170108,24737,20170107,23415,20170106,32387,20170105,37830
20170109,26718,20170108,24737,20170107,23415,20170106,32387
20170110,20143,20170109,26718,20170108,24737,20170107,23415
20170111,24287,20170110,20143,20170109,26718,20170108,24737
20170112,23526,20170111,24287,20170110,20143,20170109,26718


---
> R-043: レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [47]:
df_sales_summary <- df_customer[c("customer_id", "gender_cd", "birth_day" , "age")] %>%
    mutate(era = trunc(age / 10) * 10) %>%
    inner_join(df_receipt, by="customer_id") %>%
    group_by(gender_cd, era) %>%
    summarise(sum_amount=sum(amount), .groups = 'drop') %>%
    spread(gender_cd, sum_amount, fill=0) %>%
    rename(male='0', female='1', unknown='9')

df_sales_summary

era,male,female,unknown
<dbl>,<dbl>,<dbl>,<dbl>
10,1591,149836,4317
20,72940,1363724,44328
30,177322,693047,50441
40,19355,9320791,483512
50,54320,6685192,342923
60,272469,987741,71418
70,13435,29764,2427
80,46360,262923,5111
90,0,6260,0


---
> R-044: 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [48]:
gather(df_sales_summary, key = gender_cd, value = sum_amount, male, female, unknown) %>%
    mutate(gender_cd = case_when(
        gender_cd == "male"  ~ "00",
        gender_cd == "female"  ~ "01",
        gender_cd == "unknown"  ~ "99",
    ))

era,gender_cd,sum_amount
<dbl>,<chr>,<dbl>
10,00,1591
20,00,72940
30,00,177322
40,00,19355
50,00,54320
60,00,272469
70,00,13435
80,00,46360
90,00,0


---
> R-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [49]:
df_tmp <- cbind(df_customer['customer_id'], strftime(df_customer$birth_day, format="%Y%m%d"))
colnames(df_tmp) <- c("customer_id","birth_day")
head(df_tmp,10)

,customer_id,birth_day
,<chr>,<fct>
1,CS021313000114,19810429
2,CS037613000071,19520401
3,CS031415000172,19761004
4,CS028811000001,19330327
5,CS001215000145,19950329
6,CS020401000016,19740915
7,CS015414000103,19770809
8,CS029403000008,19730817
9,CS015804000004,19310502


---
> R-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [50]:
df_tmp <- cbind(df_customer["customer_id"], 
                strptime(df_customer$application_date, '%Y%m%d'))
colnames(df_tmp) <- c("customer_id","application_date")
head(df_tmp, 10)

,customer_id,application_date
,<chr>,<dttm>
1,CS021313000114,2015-09-05
2,CS037613000071,2015-04-14
3,CS031415000172,2015-05-29
4,CS028811000001,2016-01-15
5,CS001215000145,2017-06-05
6,CS020401000016,2015-02-25
7,CS015414000103,2015-07-22
8,CS029403000008,2015-05-15
9,CS015804000004,2015-06-07


---
> R-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [51]:
df_tmp <- cbind(df_receipt[c('receipt_no', 'receipt_sub_no')], 
                strptime(as.character(df_receipt$sales_ymd), '%Y%m%d'))
colnames(df_tmp) <- c("receipt_no","receipt_sub_no", "sales_ymd")
head(df_tmp, 10)

,receipt_no,receipt_sub_no,sales_ymd
,<int>,<int>,<dttm>
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


---
> R-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（POSIXct）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [52]:
df_tmp <- cbind(df_receipt[c('receipt_no', 'receipt_sub_no')], 
                as.POSIXct(df_receipt$sales_epoch, origin="1970-01-01"))
colnames(df_tmp) <- c("receipt_no","receipt_sub_no", "sales_ymd")
head(df_tmp, 10)

,receipt_no,receipt_sub_no,sales_ymd
,<int>,<int>,<dttm>
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


---
> R-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（POSIXct）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [53]:
df_tmp <- cbind(df_receipt[c('receipt_no', 'receipt_sub_no')], 
                substring(as.POSIXct(df_receipt$sales_epoch, origin="1970-01-01"), 1, 4))
colnames(df_tmp) <- c("receipt_no","receipt_sub_no", "sales_ymd")
head(df_tmp, 10)

,receipt_no,receipt_sub_no,sales_ymd
,<int>,<int>,<fct>
1,112,1,2018
2,1132,2,2018
3,1102,1,2017
4,1132,1,2019
5,1102,2,2018
6,1112,1,2019
7,1102,2,2018
8,1102,1,2019
9,1112,2,2017


---
> R-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（POSIXct）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [54]:
df_tmp <- cbind(df_receipt[c('receipt_no', 'receipt_sub_no')], 
                substring(as.POSIXct(df_receipt$sales_epoch, origin="1970-01-01"), 6, 7))
colnames(df_tmp) <- c("receipt_no","receipt_sub_no", "sales_ymd")
head(df_tmp, 10)

,receipt_no,receipt_sub_no,sales_ymd
,<int>,<int>,<fct>
1,112,1,11
2,1132,2,11
3,1102,1,07
4,1132,1,02
5,1102,2,08
6,1112,1,06
7,1102,2,12
8,1102,1,09
9,1112,2,05


---
> R-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（POSIXct）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [55]:
df_tmp <- cbind(df_receipt[c('receipt_no', 'receipt_sub_no')], 
                substring(as.POSIXct(df_receipt$sales_epoch, origin="1970-01-01"), 9, 10))
colnames(df_tmp) <- c("receipt_no","receipt_sub_no", "sales_ymd")
head(df_tmp, 10)

,receipt_no,receipt_sub_no,sales_ymd
,<int>,<int>,<fct>
1,112,1,03
2,1132,2,18
3,1102,1,12
4,1132,1,05
5,1102,2,21
6,1112,1,05
7,1102,2,05
8,1102,1,22
9,1112,2,04


---
> R-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、合計金額とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [56]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount=sum(amount), .groups = 'drop') %>%
    mutate(bit_amount = ifelse(sum_amount <= 2000, 0, 1))　%>%
    slice(1:10)

customer_id,sum_amount,bit_amount
<chr>,<int>,<dbl>
CS001113000004,1298,0
CS001114000005,626,0
CS001115000010,3044,1
CS001205000004,1988,0
CS001205000006,3337,1
CS001211000025,456,0
CS001212000027,448,0
CS001212000031,296,0
CS001212000046,228,0


---
> R-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを０に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [57]:
inner_join(df_receipt, df_customer, by = "customer_id") %>%
    mutate(postal_bit = ifelse(100 <= as.integer(str_sub(postal_cd, start=1, end=3)) & 
                                   as.integer(str_sub(postal_cd, start=1, end=3)) <= 209, 1, 0)) %>%
    group_by(postal_bit) %>%
    summarise(customer_cnt = n_distinct(customer_id), .groups = 'drop')

postal_bit,customer_cnt
<dbl>,<int>
0,3906
1,4400


---
> R-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [58]:
df_customer %>%
    mutate(prefecture_cd = case_when(
                                str_sub(address, start=1, end=3) == "埼玉県" ~ "11",
                                str_sub(address, start=1, end=3) == "千葉県" ~ "12",
                                str_sub(address, start=1, end=3) == "東京都" ~ "13",
                                str_sub(address, start=1, end=3) == "神奈川" ~ "14")) %>%
    select(customer_id, address, prefecture_cd) %>%
    slice(1:10)

customer_id,address,prefecture_cd
<chr>,<chr>,<chr>
CS021313000114,神奈川県伊勢原市粟窪**********,14
CS037613000071,東京都江東区南砂**********,13
CS031415000172,東京都渋谷区代々木**********,13
CS028811000001,神奈川県横浜市泉区和泉町**********,14
CS001215000145,東京都大田区仲六郷**********,13
CS020401000016,東京都板橋区若木**********,13
CS015414000103,東京都江東区北砂**********,13
CS029403000008,千葉県浦安市海楽**********,12
CS015804000004,東京都江東区北砂**********,13


---
> R-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [59]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = 'drop') %>%
    mutate(amount_cat = case_when(
        sum_amount < quantile(sum_amount)[2]  ~ "1",
        sum_amount < quantile(sum_amount)[3]  ~ "2",
        sum_amount < quantile(sum_amount)[4]  ~ "3",
        quantile(sum_amount)[4] <= sum_amount ~ "4"
    )) %>%
    slice(1:10)

customer_id,sum_amount,amount_cat
<chr>,<int>,<chr>
CS001113000004,1298,2
CS001114000005,626,2
CS001115000010,3044,3
CS001205000004,1988,3
CS001205000006,3337,3
CS001211000025,456,1
CS001212000027,448,1
CS001212000031,296,1
CS001212000046,228,1


---
> R-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [60]:
df_customer[c("customer_id", "birth_day","age")] %>%
    mutate(era = trunc(age / 10) * 10) %>%
    mutate(era = case_when(
        era < 60 ~ as.factor(era),
        era >= 60 ~　as.factor("60")
    )) %>%
    slice(1:10)


customer_id,birth_day,age,era
<chr>,<date>,<int>,<fct>
CS021313000114,1981-04-29,37,30
CS037613000071,1952-04-01,66,60
CS031415000172,1976-10-04,42,40
CS028811000001,1933-03-27,86,60
CS001215000145,1995-03-29,24,20
CS020401000016,1974-09-15,44,40
CS015414000103,1977-08-09,41,40
CS029403000008,1973-08-17,45,40
CS015804000004,1931-05-02,87,60


---
> R-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [61]:
# 性別コード１桁と年代コード２桁を連結した性年代コードを生成する
df_customer[c("customer_id", "gender_cd","birth_day", "age")] %>%
    mutate(era = trunc(age / 10) * 10) %>%
    mutate(era = case_when(
        era < 60 ~ as.factor(era),
        era >= 60 ~　as.factor("60"))) %>%
    mutate(gender_era_cd = paste(gender_cd, era, sep="")) %>%
    slice(1:10)


customer_id,gender_cd,birth_day,age,era,gender_era_cd
<chr>,<chr>,<date>,<int>,<fct>,<chr>
CS021313000114,1,1981-04-29,37,30,130
CS037613000071,9,1952-04-01,66,60,960
CS031415000172,1,1976-10-04,42,40,140
CS028811000001,1,1933-03-27,86,60,160
CS001215000145,1,1995-03-29,24,20,120
CS020401000016,0,1974-09-15,44,40,040
CS015414000103,1,1977-08-09,41,40,140
CS029403000008,0,1973-08-17,45,40,040
CS015804000004,0,1931-05-02,87,60,060


---
> R-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [62]:
dummy_gender_model <- dummyVars(~gender_cd, data = df_customer, fullRank = FALSE)
dummy_dender <- predict(dummy_gender_model, df_customer)
head(cbind(df_customer['customer_id'], dummy_dender),10)

,customer_id,gender_cd0,gender_cd1,gender_cd9
,<chr>,<dbl>,<dbl>,<dbl>
1,CS021313000114,0,1,0
2,CS037613000071,0,0,1
3,CS031415000172,0,1,0
4,CS028811000001,0,1,0
5,CS001215000145,0,1,0
6,CS020401000016,1,0,0
7,CS015414000103,0,1,0
8,CS029403000008,1,0,0
9,CS015804000004,1,0,0


---
> R-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [63]:
# Rのscaleを使用しているため不偏標準偏差で標準化される
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = 'drop') %>%
    mutate(normalized_amount = scale(sum_amount, center = TRUE, scale = TRUE)) %>%
    slice(1:10)

customer_id,sum_amount,normalized_amount
<chr>,<int>,"<dbl[,1]>"
CS001113000004,1298,-0.4593502
CS001114000005,626,-0.7063478
CS001115000010,3044,0.1824025
CS001205000004,1988,-0.2057366
CS001205000006,3337,0.2900964
CS001211000025,456,-0.7688324
CS001212000027,448,-0.7717728
CS001212000031,296,-0.8276413
CS001212000046,228,-0.8526351


---
> R-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [64]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = 'drop') %>%
    mutate(normalized_amount = scale(sum_amount, center = min(sum_amount), scale = max(sum_amount) - min(sum_amount))) %>%
    slice(1:10)

customer_id,sum_amount,normalized_amount
<chr>,<int>,"<dbl[,1]>"
CS001113000004,1298,0.053354188
CS001114000005,626,0.024157108
CS001115000010,3044,0.129214460
CS001205000004,1988,0.083333333
CS001205000006,3337,0.141944734
CS001211000025,456,0.016770942
CS001212000027,448,0.016423358
CS001212000031,296,0.009819256
CS001212000046,228,0.006864790


---
> R-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ（ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること）。結果は10件表示させれば良い。

In [65]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = 'drop') %>%
    mutate(log_amount = log((sum_amount + 1),10)) %>%
    slice(1:10)

customer_id,sum_amount,log_amount
<chr>,<int>,<dbl>
CS001113000004,1298,3.113609
CS001114000005,626,2.797268
CS001115000010,3044,3.483587
CS001205000004,1988,3.298635
CS001205000006,3337,3.523486
CS001211000025,456,2.659916
CS001212000027,448,2.652246
CS001212000031,296,2.472756
CS001212000046,228,2.359835


---
> R-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [66]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = 'drop') %>%
    mutate(log_amount = log(sum_amount + 1)) %>%
    slice(1:10)

customer_id,sum_amount,log_amount
<chr>,<int>,<dbl>
CS001113000004,1298,7.169350
CS001114000005,626,6.440947
CS001115000010,3044,8.021256
CS001205000004,1988,7.595387
CS001205000006,3337,8.113127
CS001211000025,456,6.124683
CS001212000027,448,6.107023
CS001212000031,296,5.693732
CS001212000046,228,5.433722


---
> R-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [67]:
df_product %>%
    mutate(unit_profit = unit_price - unit_cost) %>%
    slice(1:10)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_profit
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>
P040101001,04,0401,040101,198,149,49
P040101002,04,0401,040101,218,164,54
P040101003,04,0401,040101,230,173,57
P040101004,04,0401,040101,248,186,62
P040101005,04,0401,040101,268,201,67
P040101006,04,0401,040101,298,224,74
P040101007,04,0401,040101,338,254,84
P040101008,04,0401,040101,420,315,105
P040101009,04,0401,040101,498,374,124


---
> R-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。 ただし、単価と原価にはNULLが存在することに注意せよ。

In [68]:
df_product %>%
    mutate(unit_profit_rate = (unit_price - unit_cost)/ unit_price)%>%
    summarise(total_mean = mean(unit_profit_rate, na.rm = TRUE))


total_mean
<dbl>
0.2491139


---
> R-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [69]:
df_product %>%
    mutate(new_price = trunc(unit_cost / 0.7)) %>%
    mutate(new_profit_rate = (new_price - unit_cost)/ new_price) %>%
    slice(1:10)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_profit_rate
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
P040101001,04,0401,040101,198,149,212,0.2971698
P040101002,04,0401,040101,218,164,234,0.2991453
P040101003,04,0401,040101,230,173,247,0.2995951
P040101004,04,0401,040101,248,186,265,0.2981132
P040101005,04,0401,040101,268,201,287,0.2996516
P040101006,04,0401,040101,298,224,320,0.3000000
P040101007,04,0401,040101,338,254,362,0.2983425
P040101008,04,0401,040101,420,315,450,0.3000000
P040101009,04,0401,040101,498,374,534,0.2996255


---
> R-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [70]:
df_product %>%
    mutate(new_price = round(unit_cost / 0.7)) %>%
    mutate(new_profit_rate = (new_price - unit_cost)/ new_price) %>%
    slice(1:10)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_profit_rate
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
P040101001,04,0401,040101,198,149,213,0.3004695
P040101002,04,0401,040101,218,164,234,0.2991453
P040101003,04,0401,040101,230,173,247,0.2995951
P040101004,04,0401,040101,248,186,266,0.3007519
P040101005,04,0401,040101,268,201,287,0.2996516
P040101006,04,0401,040101,298,224,320,0.3000000
P040101007,04,0401,040101,338,254,363,0.3002755
P040101008,04,0401,040101,420,315,450,0.3000000
P040101009,04,0401,040101,498,374,534,0.2996255


---
> R-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [71]:
df_product %>%
    mutate(new_price = ceiling(unit_cost / 0.7)) %>%
    mutate(new_profit_rate = (new_price - unit_cost)/ new_price) %>%
    slice(1:10)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_profit_rate
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
P040101001,04,0401,040101,198,149,213,0.3004695
P040101002,04,0401,040101,218,164,235,0.3021277
P040101003,04,0401,040101,230,173,248,0.3024194
P040101004,04,0401,040101,248,186,266,0.3007519
P040101005,04,0401,040101,268,201,288,0.3020833
P040101006,04,0401,040101,298,224,320,0.3000000
P040101007,04,0401,040101,338,254,363,0.3002755
P040101008,04,0401,040101,420,315,451,0.3015521
P040101009,04,0401,040101,498,374,535,0.3009346


---
> R-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [72]:
head(cbind(df_product['product_cd'],trunc(df_product['unit_price'] * 1.1)),10)

,product_cd,unit_price
,<chr>,<dbl>
1,P040101001,217
2,P040101002,239
3,P040101003,253
4,P040101004,272
5,P040101005,294
6,P040101006,327
7,P040101007,371
8,P040101008,462
9,P040101009,547


---
> R-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [73]:
inner_join(df_receipt, df_product[c("product_cd","category_major_cd")], by="product_cd") %>%
    filter(category_major_cd == "07") %>%
    group_by(customer_id) %>%
    summarise(amount_07 = sum(amount), .groups = 'drop') %>%
    inner_join(df_receipt, by="customer_id") %>%
    group_by(customer_id) %>%
    summarise(sum_amount=sum(amount), amount_07=max(amount_07), .groups = 'drop') %>%
    mutate(rate_07 = amount_07 / sum_amount) %>%
    slice(1:10)

customer_id,sum_amount,amount_07,rate_07
<chr>,<int>,<int>,<dbl>
CS001113000004,1298,1298,1.0000000
CS001114000005,626,486,0.7763578
CS001115000010,3044,2694,0.8850197
CS001205000004,1988,346,0.1740443
CS001205000006,3337,2004,0.6005394
CS001212000027,448,200,0.4464286
CS001212000031,296,296,1.0000000
CS001212000046,228,108,0.4736842
CS001212000070,456,308,0.6754386


---
> R-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [74]:
inner_join(df_receipt[c("customer_id", "sales_ymd")], 
                     df_customer[c("customer_id","application_date")], by="customer_id") %>%
    distinct(.,.keep_all=TRUE) %>%
    mutate(elapsed_days = strptime(as.character(sales_ymd), '%Y%m%d')
                            - strptime(application_date, '%Y%m%d')) %>%
    select(customer_id, sales_ymd, application_date, elapsed_days) %>%
    slice(1:10)

customer_id,sales_ymd,application_date,elapsed_days
<chr>,<int>,<chr>,<drtn>
CS006214000001,20181103,20150201,1371 days
CS008415000097,20181118,20150322,1337 days
CS028414000014,20170712,20150711,732 days
CS025415000050,20180821,20160131,933 days
CS003515000195,20190605,20150306,1552 days
CS024514000042,20181205,20151010,1152 days
CS040415000178,20190922,20150627,1548 days
CS027514000015,20191010,20151101,1439 days
CS025415000134,20190918,20150720,1521 days


---
> R-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [75]:
(time_length(strptime('20171122', '%Y%m%d') - strptime('20150623', '%Y%m%d'), "month"))

[1] 29.01027

In [76]:
time_length(interval(strptime('20150623', '%Y%m%d'), strptime('20171122', '%Y%m%d')), "month")

[1] 28.96774

In [77]:
inner_join(df_receipt[c("customer_id", "sales_ymd")], 
                     df_customer[c("customer_id","application_date")], by="customer_id") %>%
    distinct(.,.keep_all=TRUE) %>%
    mutate(elapsed_months = trunc(time_length(
                                        interval(
                                            strptime(application_date, '%Y%m%d'),
                                            strptime(as.character(sales_ymd), '%Y%m%d')
                                        ), "month")
                                 )) %>%
    select(customer_id, sales_ymd, application_date, elapsed_months) %>%
    slice(1:10)

customer_id,sales_ymd,application_date,elapsed_months
<chr>,<int>,<chr>,<dbl>
CS006214000001,20181103,20150201,45
CS008415000097,20181118,20150322,43
CS028414000014,20170712,20150711,24
CS025415000050,20180821,20160131,30
CS003515000195,20190605,20150306,50
CS024514000042,20181205,20151010,37
CS040415000178,20190922,20150627,50
CS027514000015,20191010,20151101,47
CS025415000134,20190918,20150720,49


---
> R-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [78]:
inner_join(df_receipt[c("customer_id", "sales_ymd")], 
                     df_customer[c("customer_id","application_date")], by="customer_id") %>%
    distinct(.,.keep_all=TRUE) %>%
    mutate(elapsed_years = trunc(time_length(
                                        interval(
                                            strptime(application_date, '%Y%m%d'),
                                            strptime(as.character(sales_ymd), '%Y%m%d')
                                        ), "year")
                                  ))%>%
    select(customer_id, sales_ymd, application_date, elapsed_years) %>%
    slice(1:10)

customer_id,sales_ymd,application_date,elapsed_years
<chr>,<int>,<chr>,<dbl>
CS006214000001,20181103,20150201,3
CS008415000097,20181118,20150322,3
CS028414000014,20170712,20150711,2
CS025415000050,20180821,20160131,2
CS003515000195,20190605,20150306,4
CS024514000042,20181205,20151010,3
CS040415000178,20190922,20150627,4
CS027514000015,20191010,20151101,3
CS025415000134,20190918,20150720,4


---
> R-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [79]:
inner_join(df_receipt[c("customer_id", "sales_ymd")], 
                     df_customer[c("customer_id","application_date")], by="customer_id") %>%
    distinct(.,.keep_all=TRUE) %>%
    mutate(elapsed_epoch = as.numeric(strptime(as.character(sales_ymd), '%Y%m%d'))
                            - as.numeric(strptime(application_date, '%Y%m%d'))) %>%
    select(customer_id, sales_ymd, application_date, elapsed_epoch) %>%
    slice(1:10)

customer_id,sales_ymd,application_date,elapsed_epoch
<chr>,<int>,<chr>,<dbl>
CS006214000001,20181103,20150201,118454400
CS008415000097,20181118,20150322,115516800
CS028414000014,20170712,20150711,63244800
CS025415000050,20180821,20160131,80611200
CS003515000195,20190605,20150306,134092800
CS024514000042,20181205,20151010,99532800
CS040415000178,20190922,20150627,133747200
CS027514000015,20191010,20151101,124329600
CS025415000134,20190918,20150720,131414400


---
> R-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、顧客ID、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [80]:
df_receipt[c("customer_id", "sales_ymd")] %>%
    distinct(.,.keep_all=TRUE) %>%
    # 以下では開始日が日曜日となるため1日前シフトさせることで日曜日を前週の最終日に変換している。
    # その状態でfloor_dateをすると、週の開始日として前週の日曜日が取得できるため、開始日を月曜日にするために１を足している。
    mutate(monday = as.Date(floor_date(strptime(as.character(sales_ymd), '%Y%m%d') - 1 , unit="week")) + 1) %>%
    mutate(elapse_weekday = as.Date(strptime(as.character(sales_ymd), '%Y%m%d')) - monday) %>%
    select(customer_id, sales_ymd, monday, elapse_weekday) %>%
    slice(1:10)

customer_id,sales_ymd,monday,elapse_weekday
<chr>,<int>,<date>,<drtn>
CS006214000001,20181103,2018-10-29,5 days
CS008415000097,20181118,2018-11-12,6 days
CS028414000014,20170712,2017-07-10,2 days
ZZ000000000000,20190205,2019-02-04,1 days
CS025415000050,20180821,2018-08-20,1 days
CS003515000195,20190605,2019-06-03,2 days
CS024514000042,20181205,2018-12-03,2 days
CS040415000178,20190922,2019-09-16,6 days
ZZ000000000000,20170504,2017-05-01,3 days


---
> R-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [81]:
head(sample_frac(tbl = df_customer, 0.01),10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS007312000135,浜本 美菜,1,女性,1980-02-27,39,276-0028,千葉県八千代市村上**********,S12007,20150717,0-00000000-0
2,CS034412000050,宮原 綾,1,女性,1970-03-19,49,213-0022,神奈川県川崎市高津区千年**********,S14034,20150618,0-00000000-0
3,CS001602000021,長瀬 有起哉,0,男性,1950-04-05,68,212-0058,神奈川県川崎市幸区鹿島田**********,S13001,20150725,0-00000000-0
4,CS038513000010,井田 涼子,1,女性,1966-03-15,53,279-0041,千葉県浦安市堀江**********,S13038,20150712,7-20100404-A
5,CS010215000008,細川 瞳,1,女性,1989-09-23,29,230-0075,神奈川県横浜市鶴見区上の宮**********,S14010,20150922,0-00000000-0
6,CS016515000226,波多野 ちえみ,9,不明,1963-12-30,55,184-0004,東京都小金井市本町**********,S13016,20150704,E-20100730-D
7,CS029514000049,長田 美佐子,9,不明,1966-11-30,52,279-0042,千葉県浦安市東野**********,S12029,20150131,0-00000000-0
8,CS002411000146,加藤 由宇,1,女性,1970-06-28,48,187-0031,東京都小平市小川東町**********,S13002,20170909,1-20100609-1
9,CS022512000056,原 はるみ,1,女性,1964-10-11,54,240-0112,神奈川県三浦郡葉山町堀内**********,S14022,20151103,2-20100725-3


---
> R-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [82]:
set.seed(71)
df_customer %>%
    group_by(gender_cd) %>%
    sample_frac(0.1) %>%
    summarise(customer_num = n(), .groups = 'drop')

gender_cd,customer_num
<chr>,<int>
0,298
1,1792
9,107


---
> R-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [83]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = 'drop') %>%
    filter(abs(sum_amount - mean(sum_amount)) / sd(sum_amount) >= 3) %>%
    slice(1:10)

customer_id,sum_amount
<chr>,<int>
CS001605000009,18925
CS006415000147,12723
CS006515000023,18372
CS006515000125,12575
CS006515000209,11373
CS007115000006,11528
CS007514000056,13293
CS007514000094,15735
CS007515000107,11188


---
> R-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [84]:
# 確認用コード
df_tmp <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = 'drop')

quantile(df_tmp$sum_amount)

0%      25%      50%      75%     100% 
   70.00   548.25  1478.00  3649.75 23086.00

In [85]:
# 確認用コード
(3602-554) * 1.5 + 3602

[1] 8174

In [86]:
df_receipt %>%
    group_by(customer_id) %>%
    filter(!grepl("^Z", customer_id)) %>%
    summarise(sum_amount = sum(amount), .groups = 'drop') %>%
    filter(sum_amount < quantile(sum_amount)[2] - 1.5 * (quantile(sum_amount)[4] - quantile(sum_amount)[2]) |
           sum_amount > quantile(sum_amount)[4] + 1.5 * (quantile(sum_amount)[4] - quantile(sum_amount)[2])) %>%
    slice(1:10)

customer_id,sum_amount
<chr>,<int>
CS001414000048,8584
CS001605000009,18925
CS002415000594,9568
CS004414000181,9584
CS005415000137,8734
CS006414000001,9156
CS006414000029,9179
CS006415000105,10042
CS006415000147,12723


---
> R-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [87]:
sapply(df_product, function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 7                  7

---
> R-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [88]:
nrow(df_product)
df_product_1 <- na.omit(df_product)
nrow(df_product_1)

[1] 10030

[1] 10023

---
> R-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [89]:
price_mean <- round(mean(df_product$unit_price, na.rm=TRUE))
cost_mean <- round(mean(df_product$unit_cost, na.rm=TRUE))

df_product_2 <- df_product %>%
    replace_na(list(unit_price = price_mean, unit_cost = cost_mean))
    
sapply(df_product_2, function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 0                  0

---
> R-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [90]:
price_median <- round(median(df_product$unit_price, na.rm=TRUE))
cost_median <- round(median(df_product$unit_cost, na.rm=TRUE))

df_product_3 <- df_product %>%
    replace_na(list(unit_price = price_median, unit_cost = cost_median))
    
sapply(df_product_3, function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 0                  0

---
> R-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [91]:
df_product_4 <- df_product %>%
    group_by(category_small_cd) %>%
    summarise(price_median = round(median(unit_price, na.rm=TRUE)), 
              cost_median=round(median(unit_cost, na.rm=TRUE)), .groups = 'drop') %>%
    inner_join(df_product, by="category_small_cd") %>%
    mutate(unit_price = ifelse(is.na(unit_price), price_median, unit_price),
           unit_cost = ifelse(is.na(unit_cost), cost_median, unit_cost))

sapply(df_product_4, function(x) sum(is.na(x)))

category_small_cd       price_median        cost_median         product_cd 
                 0                  0                  0                  0 
 category_major_cd category_medium_cd         unit_price          unit_cost 
                 0                  0                  0                  0

In [92]:
df_product_4 %>%
    filter(product_cd == 'P050103021')

category_small_cd,price_median,cost_median,product_cd,category_major_cd,category_medium_cd,unit_price,unit_cost
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
050103,132,100,P050103021,05,0501,132,100


---
> R-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [93]:
df_receipt_2019 <- df_receipt %>%
    filter(20190101 <= sales_ymd & sales_ymd <= 20191231)

df_amount_all <-left_join(df_customer, df_receipt, by="customer_id") %>%
                    group_by(customer_id) %>%
                    summarise(amount_all = sum(amount), .groups = 'drop')

df_amount_2019 <-left_join(df_customer, df_receipt_2019, by="customer_id") %>%
                    group_by(customer_id) %>%
                    summarise(amount_2019 = sum(amount), .groups = 'drop') %>%
                    inner_join(df_amount_all, by="customer_id") %>%
                    replace_na(list(amount_2019 = 0, amount_all = 0)) %>%
                    mutate(amount_rate = ifelse(amount_all == 0, 0, amount_2019 / amount_all))

df_amount_2019 %>%
    filter(amount_rate > 0) %>%
    slice(1:10)

customer_id,amount_2019,amount_all,amount_rate
<chr>,<dbl>,<dbl>,<dbl>
CS001113000004,1298,1298,1.0000000
CS001114000005,188,626,0.3003195
CS001115000010,578,3044,0.1898817
CS001205000004,702,1988,0.3531187
CS001205000006,486,3337,0.1456398
CS001211000025,456,456,1.0000000
CS001212000070,456,456,1.0000000
CS001214000009,664,4685,0.1417289
CS001214000017,2962,4132,0.7168441


In [94]:
sapply(df_amount_2019, function(x) sum(is.na(x)))

customer_id amount_2019  amount_all amount_rate 
          0           0           0           0

---
> R-085: 郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。

In [95]:
df_customer_1 <- inner_join(df_customer[c("customer_id", "postal_cd")], 
           df_geocode[c("postal_cd", "longitude" ,"latitude")], by="postal_cd") %>%
    group_by(customer_id) %>%
    summarise(m_longiture=mean(longitude),m_latitude=mean(latitude), .groups = 'drop') %>%
    inner_join(df_customer, by="customer_id")

head(df_customer_1, 5)

customer_id,m_longiture,m_latitude,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
CS001105000001,139.7024,35.54137,中島 利夫,0,男性,2000-01-14,19,144-0056,東京都大田区西六郷**********,S13001,20170310,0-00000000-0
CS001112000009,139.7039,35.58670,秦 美里,1,女性,2006-08-24,12,143-0026,東京都大田区西馬込**********,S13001,20150703,0-00000000-0
CS001112000019,139.7469,35.57153,門脇 莉沙,1,女性,2001-01-31,18,143-0004,東京都大田区昭和島**********,S13001,20170207,0-00000000-0
CS001112000021,139.7024,35.54137,長澤 麗奈,1,女性,2001-12-15,17,144-0056,東京都大田区西六郷**********,S13001,20170612,0-00000000-0
CS001112000023,139.7469,35.57153,戸田 一恵,1,女性,2004-01-26,15,143-0004,東京都大田区昭和島**********,S13001,20170724,0-00000000-0


---
> R-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [96]:
calc_distance <- function(x1, y1, x2, y2) {
    distance <- 6371 * acos(sin(y1 * pi / 180) * sin(y2 * pi / 180) 
                    + cos(y1 * pi / 180) * cos(y2 * pi / 180) * cos((x1 * pi / 180) - (x2 * pi / 180 )))
    return(distance)
}

inner_join(df_customer_1, df_store, by = c("application_store_cd" = "store_cd")) %>%
    mutate(distance = calc_distance(m_longiture, m_latitude, longitude, latitude)) %>%
    select(customer_id, address.x, address.y, distance) %>%
    slice(1:10)

customer_id,address.x,address.y,distance
<chr>,<chr>,<chr>,<dbl>
CS001105000001,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479789
CS001112000009,東京都大田区西馬込**********,東京都大田区仲六郷二丁目,4.020495
CS001112000019,東京都大田区昭和島**********,東京都大田区仲六郷二丁目,3.783015
CS001112000021,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479789
CS001112000023,東京都大田区昭和島**********,東京都大田区仲六郷二丁目,3.783015
CS001112000024,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479789
CS001112000029,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479789
CS001112000030,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479789
CS001113000004,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479789


---
> R-087: 顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [97]:
df_sales_amount <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount=sum(amount), .groups = 'drop')

df_customer_u <- left_join(df_customer, df_sales_amount, by="customer_id") %>%
    arrange(desc(sum_amount), customer_id) %>%
    distinct(customer_name, postal_cd, .keep_all=TRUE) 

#減少件数をカウント
nrow(df_customer) - nrow(df_customer_u)

[1] 30

---
> R-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [98]:
df_customer_n = inner_join(df_customer, df_customer_u[c("customer_id", "customer_name", "postal_cd")],
               by=c("customer_name", "postal_cd")) %>%
           rename(integration_id = customer_id.y)
# 確認
length(unique(df_customer_n$customer_id)) - length(unique(df_customer_n$integration_id))

[1] 30

---
> R-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [99]:
#rsampleのinitial_splitを使った例
df_sales_amount <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = 'drop')

df_customer_s <- inner_join(df_customer, df_sales_amount, by="customer_id")

df_split <- initial_split(df_customer_s, prop = 0.8)

df_customer_train <- training(df_split)
df_customer_test <- testing(df_split)

# 確認
print(nrow(df_customer_train))
print(nrow(df_customer_test))

[1] 6645
[1] 1661


---
> R-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [100]:
#caretのcreateTimeSlicesを使った例
df_ts_amount <- df_receipt %>%
    group_by(substr(as.character(sales_ymd), 1, 6)) %>%
    summarise(sum_amount = sum(amount), .groups = 'drop')
colnames(df_ts_amount) <- c("sales_ym","sales_amount")

timeSlice <- createTimeSlices(1:nrow(df_ts_amount), initialWindow = 12, horizon = 6, skip = 5, fixedWindow = TRUE)

df_train_1 <- df_ts_amount[timeSlice$train[[1]],]
df_train_2 <- df_ts_amount[timeSlice$train[[2]],]
df_train_3 <- df_ts_amount[timeSlice$train[[3]],]

df_test_1 <- df_ts_amount[timeSlice$test[[1]],]
df_test_2 <- df_ts_amount[timeSlice$test[[2]],]
df_test_3 <- df_ts_amount[timeSlice$test[[3]],]

---
> R-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [101]:
#recipesパッケージを使った例
#themis::step_downsample()を使うようワーニングが出るが、なるべくインストールパッケージを少なくするためrecipeのstep_downsample()を使用
df_sales_amount <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = 'drop')
    
df_tmp <- left_join(df_customer, df_sales_amount, by="customer_id") %>%
    replace_na(list(sum_amount = 0)) %>%
    mutate(sum_amount = factor(ifelse(sum_amount > 0, 1, 0)))

set.seed(71)

df_down_train <- df_tmp %>% 
  recipe(~ .) %>% 
  step_downsample(sum_amount) %>%
  prep() %>% 
  juice()

df_down_train %>%
    group_by(sum_amount) %>%
    summarise(cnt = n(), .groups = 'drop')

Warning message:
“`step_downsample()` is deprecated as of recipes 0.1.13.
Please use `themis::step_downsample()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”


sum_amount,cnt
<fct>,<int>
0,8306
1,8306


In [102]:
head(df_down_train, 3)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,sum_amount
<fct>,<fct>,<fct>,<fct>,<date>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
CS018311000038,尾崎 倫子,1,女性,1985-11-04,33,359-0035,埼玉県所沢市西新井町**********,S13018,20150921,0-00000000-0,0
CS009513000243,芦田 莉央,1,女性,1966-04-30,52,158-0093,東京都世田谷区上野毛**********,S13009,20180221,0-00000000-0,0
CS029503000054,日下部 哲平,0,男性,1965-07-10,53,279-0012,千葉県浦安市入船**********,S12029,20160518,0-00000000-0,0


In [103]:
# #unbalancedのubUnderを使った例（unbalamncedの依存関係にあるxmlパッケージがR3.6.3でインストールできなくなっているためコメントアウト中※2020.07.29時点）
# #unbalancedも良く利用されるのでコードは残しておく

# df_sales_amount <- df_receipt %>%
#     group_by(customer_id) %>%
#     summarise(sum_amount = sum(amount))
    
# df_tmp <- left_join(df_customer, df_sales_amount, by="customer_id") %>%
#     replace_na(list(sum_amount = 0)) %>%
#     mutate(sum_amount = ifelse(sum_amount > 0, 1, 0))

# data_under<-ubUnder(X=df_tmp[,colnames(df_tmp) != "sum_amount"], Y= df_tmp$sum_amount, perc = 50, method = "percPos")

# df_down_sampling <- cbind(data_under$X, data_under$Y) %>%
#     rename(sum_amount = 'data_under$Y')

---
> R-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [104]:
df_gender = unique(df_customer[c("gender_cd", "gender")])
df_customer_s = df_customer[, colnames(df_customer) != "gender"]

---
> R-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [105]:
df_product_full <- inner_join(df_product, df_category[c("category_small_cd", 
                                                        "category_major_name",
                                                        "category_medium_name",
                                                        "category_small_name")], by="category_small_cd")

---
> R-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [106]:
write.csv(df_product_full, "../data/R_df_product_full_UTF-8_header.csv", fileEncoding="UTF-8")

---
> R-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [107]:
write.csv(df_product_full, "../data/R_df_product_full_CP932_header.csv", fileEncoding="CP932")

---
> R-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata
配下とする。
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [108]:
write.table(df_product_full, "../data/R_df_product_full_UTF-8_noh.csv", col.names=FALSE, sep=",", fileEncoding="UTF-8")

---
> R-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [109]:
df_product_tmp <- read.csv('../data/R_df_product_full_UTF-8_header.csv', fileEncoding = "UTF-8")
head(df_product_tmp, 10)

,X,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,<int>,<fct>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>
1,1,P040101001,4,401,40101,198,149,惣菜,御飯類,弁当類
2,2,P040101002,4,401,40101,218,164,惣菜,御飯類,弁当類
3,3,P040101003,4,401,40101,230,173,惣菜,御飯類,弁当類
4,4,P040101004,4,401,40101,248,186,惣菜,御飯類,弁当類
5,5,P040101005,4,401,40101,268,201,惣菜,御飯類,弁当類
6,6,P040101006,4,401,40101,298,224,惣菜,御飯類,弁当類
7,7,P040101007,4,401,40101,338,254,惣菜,御飯類,弁当類
8,8,P040101008,4,401,40101,420,315,惣菜,御飯類,弁当類
9,9,P040101009,4,401,40101,498,374,惣菜,御飯類,弁当類


---
> R-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [110]:
df_product_tmp <- read.csv('../data/R_df_product_full_UTF-8_noh.csv', header=FALSE, fileEncoding="UTF-8")
head(df_product_tmp, 10)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
,<int>,<fct>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>
1,1,P040101001,4,401,40101,198,149,惣菜,御飯類,弁当類
2,2,P040101002,4,401,40101,218,164,惣菜,御飯類,弁当類
3,3,P040101003,4,401,40101,230,173,惣菜,御飯類,弁当類
4,4,P040101004,4,401,40101,248,186,惣菜,御飯類,弁当類
5,5,P040101005,4,401,40101,268,201,惣菜,御飯類,弁当類
6,6,P040101006,4,401,40101,298,224,惣菜,御飯類,弁当類
7,7,P040101007,4,401,40101,338,254,惣菜,御飯類,弁当類
8,8,P040101008,4,401,40101,420,315,惣菜,御飯類,弁当類
9,9,P040101009,4,401,40101,498,374,惣菜,御飯類,弁当類


---
> R-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [111]:
write.table(df_product_full, "../data/R_df_product_full_UTF-8_head.tsv", sep="\t", fileEncoding="UTF-8")

---
> R-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [112]:
df_product_tmp <- read.table("../data/R_df_product_full_UTF-8_head.tsv", fileEncoding="UTF-8")
head(df_product_tmp,10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,<fct>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>
1,P040101001,4,401,40101,198,149,惣菜,御飯類,弁当類
2,P040101002,4,401,40101,218,164,惣菜,御飯類,弁当類
3,P040101003,4,401,40101,230,173,惣菜,御飯類,弁当類
4,P040101004,4,401,40101,248,186,惣菜,御飯類,弁当類
5,P040101005,4,401,40101,268,201,惣菜,御飯類,弁当類
6,P040101006,4,401,40101,298,224,惣菜,御飯類,弁当類
7,P040101007,4,401,40101,338,254,惣菜,御飯類,弁当類
8,P040101008,4,401,40101,420,315,惣菜,御飯類,弁当類
9,P040101009,4,401,40101,498,374,惣菜,御飯類,弁当類


# これで１００本終わりです。おつかれさまでした！